
### Transformer의 Self Attention Mechanism은 3가지 요소 (Key / Query / Value)로 이루어져 있다.
- 각 인풋마다 고유의 key / query / value를 가지고 있다.
- 나의 query를 남의 key와 inner product 함으로서 얻어진 값이 나 -> 남 점수가 된다.
- 나의 query로 얻어진 나 -> 남 점수들에 남의 value를 곱하여 인풋 개수만큼의 벡터를 얻는다.
- 나의 query로 얻어진 벡터들을 모두 element wise sum하여 얻어진 벡터가 나의 output이 된다.
<br/><br/>

### Mechanism을 이해하는데 헷갈리는 부분들을 미리 짚고 넘어가보자.
- Key와 Query 의 길이는 항상 동일하다.
- Output과 Value의 길이는 동일하다.
- 그러나 Key & Input & Ouput의 길이는 모두 달라도 상관없다.
<br/><br/>

##### 기존 RNN 모델의 경우
- Input (X1,X2,X3,X4 .. Xn) 데이터가 고정된 벡터 C로 Encoding 된 후, 첫 Output Y1을 예측한다.
- 그 후, Y2 예측에는 C와 Y1이, Y3의 예측에는 C와 Y1,Y2가 사용된다.
- 즉 Input이 하나의 고정된 벡터로 인코딩 된 후, 예측을 거듭할 수록 영향력을 잃게 된다.

##### Attention이 사용된 Transformer의 경우
- 각 Output Y(i)에 대해서 Attention이 적용된 서로 다른 Input Encoding C(i)를 얻게 된다
<br/><br/>

### Encoder and Decoder Stacks
`Encoder`: The encoder is composed of a stack of N = 6 identical layers. Each layer has two sub-layers. The first is a multi-head self-attention mechanism, and the second is a simple, positionwise fully connected feed-forward network.

##### Positional Encoding
- input sequence의 단어 순서를 고려하기 위해서 추가된 Layer
- non-trainable vector로서 position마다 정해진 형태의 값을 더한다
- What is the positional encoding in the transformer model?
<br/><br/>

##### Self-Attention Layer
- Input의 어떤 단어에 더 초점을 맞추어 Encdoing을 진행할 지 결정하게 되는 Layer
- Multi-head로 본다는 것은 Convolution을 multi-channel로 하는 것과 같은 개념이다.
<br/><br/>

##### Pointwise Feed Forward Layer
- Transformer에서는 residual connection을 적용한 두 층의 linear layer 사용
- We employ a residual connection around each of the two sub-layers, followed by layer normalization. That is, the output of each sub-layer is LayerNorm(x + Sublayer(x)), where Sublayer(x) is the function implemented by the sub-layer itself. To facilitate these residual connections, all sub-layers in the model, as well as the embedding layers, produce outputs of dimension dmodel = 512.
<br/><br/>

`Decoder`: The decoder is also composed of a stack of N = 6 identical layers. In addition to the two sub-layers in each encoder layer, the decoder inserts a third sub-layer, which performs multi-head attention over the output of the encoder stack. Similar to the encoder, we employ residual connections around each of the sub-layers, followed by layer normalization.

##### Layer normalization
- layer normalization과 일반적으로 사용되는 batch normalization은 구별해야한다.
- 식과 같이 batch normalization이 여러 example의 동일한 feature에 대한 정규화라고 한다면,
- layer normalization은 하나의 example 내에서 여러 feature들을 정규화하는 것이다.
<br/><br/>

##### Masking
- Encoding의 경우, 모든 input을 동시에 집어넣어서 병렬적으로 수행되지만, Decoding을 그런 방법으로 수행했을 때에는 아직 문장에서 등장하지 않은 미래의 단어를 참고하게 된다. 따라서 Self-Attention을 수행할 때, 이미 번역을 수행한 Output들에 대해서만 Score를 계산한다.
<br/><br/>

### Attention
##### Scaled Dot-Product Attention
- key / query / value를 통해서 self-attention을 수행하는 방법이다.
1. key 길이에 대한 제약을 주고자 dot - product의 값을 벡터 길이에 대응하는 값으로 나눠준다.
2. 그 이후 input 길이만큼의 (value*query) 값들을 softmax를 통해 0과1 사이의 수로 바꿔준다.
3. 마지막으로 고유의 value 값들에 softmax 결과물을 곱해줌으로서 Self-Attention이 끝난다.
<br/><br/>

##### Multi-head Attention
- 문장의 word간의 다양한 관계를 capture하기 위해 위의 Self-Attention을 여러 겹으로 동시에 수행한다.
- 논문에서는 8개의 multi-head를 사용했다. 따라서 8쌍의 (K/Q/V)에 대한 initialization이 필요하다.
- Feed-Forward 이후에 8개의 output vector를 concat한 후, Linear Layer에 태워 원래 벡터 크기로 줄여준다.

<br/><br/><br/>
# 구현
- 논문에서는 Transformer 모델을 machine translation tasks 를 해결하는데 사용하였지만, Tutorial에서는 그보다는 비교적 간단한 language modeling tasks에 적용한다. Language Modeling Task는 문장의 다음 단어가 무엇일지 예측하는 과제이다.

- Language Model의 Output을 얻는 과정은 Input과 유사한 차원 크기로 Decoding하는 것이 아니라 단순히 Linear Layer를 태우는 것이다. 따라서 논문에서 사용됬던 Encoder-Decoder Attention / Masked Self Attention과 같은 여러 기교들을 사용할 필요 없이 간단하게 모델을 구현하는 것이 가능하다. 물론 그렇다고 이것의 구현이 쉽지만은 않다. 침착하고 정교한 복붙을 통해 한 줄 한 줄 이해해보려 한다.

In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.nn import TransformerEncoder, TransformerEncoderLayer

In [10]:
class TransformerModel(nn.Module):
    
    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weights()
        
        
    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask
    
    
    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)
    
    
    def forward(self, src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, self.src_mask)
        output = self.decoder(output)
        return output

In [4]:
torch.triu(torch.ones(3, 3)) == 1

tensor([[ True,  True,  True],
        [False,  True,  True],
        [False, False,  True]])

In [8]:
mask = (torch.triu(torch.ones(3, 3)) == 1).transpose(0, 1)
mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
mask

tensor([[0., -inf, -inf],
        [0., 0., -inf],
        [0., 0., 0.]])

<br/><br/><br/>
Tutorial에서 Transformer의 구조가 논문과 다른 부분들이 조금 있는데, Task가 다르기 때문에 발생한 차이이다. 우선 Output이 가장 등장 확률이 높은 단어 하나 이기 때문에 Decoder의 형태를 띄지 않고 Linear Layer 한 층으로 간소화됬다. 또한 언어 모델링 과제를 위해서는 Self - Attention 과정에서 이전 포지션의 단어들만 참조하도록 뒤의 단어들에 대한 attention mask를 사용한다.

여러 Object Method들을 한 눈에 보기에 너무 많으니 객체 속성들과 함수들을 하나씩 봐보자.

#### TransformerModel modules
1. PositionalEncoding(ninp, dropout)
2. TransformerEncoderLayer(ninp, nhead, nhid, dropout)
3.  _generate_square_subsequent_mask(self, sz)
4. nn.Embedding(ntoken, ninp)

### 1. PositionalEncoding(ninp, dropout)

In [11]:
class PositionalEncoding(nn.Module):
    
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

Positional Encoding은 embeded된 input에 고정된 값을 더해주는 모듈이다.
- max_len은 들어올 수 있는 input sequnce의 최대 길이이다.
- d_model은 input의 feature dimension이다.

In [58]:
# 예제에 적용되어 있는 Positional Encoding 클래스의 div_term을 조금 더 단순하게 (논문과 동일하게) 표현
max_len = 10
d_model = 6

pe = torch.zeros(max_len, d_model)
position = torch.arange(0, max_len, dtype = torch.float).unsqueeze(1)
div_term = torch.pow(10000, torch.arange(0,d_model,2).float()/d_model)

pe[:, 0::2] = torch.sin(position / div_term)
pe[:, 1::2] = torch.cos(position / div_term)

print(div_term)
print(pe)

tensor([  1.0000,  21.5443, 464.1590])
tensor([[ 0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000],
        [ 0.8415,  0.5403,  0.0464,  0.9989,  0.0022,  1.0000],
        [ 0.9093, -0.4161,  0.0927,  0.9957,  0.0043,  1.0000],
        [ 0.1411, -0.9900,  0.1388,  0.9903,  0.0065,  1.0000],
        [-0.7568, -0.6536,  0.1846,  0.9828,  0.0086,  1.0000],
        [-0.9589,  0.2837,  0.2300,  0.9732,  0.0108,  0.9999],
        [-0.2794,  0.9602,  0.2749,  0.9615,  0.0129,  0.9999],
        [ 0.6570,  0.7539,  0.3192,  0.9477,  0.0151,  0.9999],
        [ 0.9894, -0.1455,  0.3629,  0.9318,  0.0172,  0.9999],
        [ 0.4121, -0.9111,  0.4057,  0.9140,  0.0194,  0.9998]])


In [59]:
pe = torch.zeros(max_len, d_model)
pe

tensor([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])

In [60]:
position = torch.arange(0, max_len, dtype = torch.float).unsqueeze(1)
position

tensor([[0.],
        [1.],
        [2.],
        [3.],
        [4.],
        [5.],
        [6.],
        [7.],
        [8.],
        [9.]])

In [61]:
div_term = torch.pow(10000, torch.arange(0,d_model,2).float()/d_model)

print(torch.arange(0, d_model, 2).float()/d_model)
print(div_term)

tensor([0.0000, 0.3333, 0.6667])
tensor([  1.0000,  21.5443, 464.1590])


In [62]:
pe[:, 0::2] = torch.sin(position / div_term) # 0번 부터 2칸씩
pe[:, 1::2] = torch.cos(position / div_term) # 1번 부터 2칸씩
pe

tensor([[ 0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000],
        [ 0.8415,  0.5403,  0.0464,  0.9989,  0.0022,  1.0000],
        [ 0.9093, -0.4161,  0.0927,  0.9957,  0.0043,  1.0000],
        [ 0.1411, -0.9900,  0.1388,  0.9903,  0.0065,  1.0000],
        [-0.7568, -0.6536,  0.1846,  0.9828,  0.0086,  1.0000],
        [-0.9589,  0.2837,  0.2300,  0.9732,  0.0108,  0.9999],
        [-0.2794,  0.9602,  0.2749,  0.9615,  0.0129,  0.9999],
        [ 0.6570,  0.7539,  0.3192,  0.9477,  0.0151,  0.9999],
        [ 0.9894, -0.1455,  0.3629,  0.9318,  0.0172,  0.9999],
        [ 0.4121, -0.9111,  0.4057,  0.9140,  0.0194,  0.9998]])

transformer_architecture_positional_encoding라는 kzaemnejad 씨의 블로그 포스트에 다양한 기법들이 소개되어 있는데, Transforemer에 사용된 sin / cos 기반의 positional encoding의 장점은 
1. input dimension과 동일한 크기의 벡터로 생성 가능하고, 
2. 모델이나 Input의 형태와 무관하게 고정된 값을 갖는데 있다. 또한 div_term을 통해서 input dimension이 매우 길었을 때, positional encoding이 모델에 너무 크게 관여하는 것을 방지한다.

### 2. TransformerEncoderLayer(ninp, nhead, nhid, dropout)

In [64]:
class TransformerEncoderLayer(Module):
    def __init__(self, 
                 d_model, 
                 nhead, 
                 dim_feedforward=2048, 
                 dropout=0.1, 
                 activation="relu"):
        
        super(TransformerEncoderLayer, self).__init__()
        
        self.self_attn = MultiheadAttention(d_model, nhead, dropout=dropout)
        
        # Implementation of Feedforward model
        self.linear1 = Linear(d_model, dim_feedforward)
        self.dropout = Dropout(dropout)
        self.linear2 = Linear(dim_feedforward, d_model)

        self.norm1 = LayerNorm(d_model)
        self.norm2 = LayerNorm(d_model)
        self.dropout1 = Dropout(dropout)
        self.dropout2 = Dropout(dropout)

        self.activation = _get_activation_fn(activation)

	def forward(self, 
                src: Tensor, 
                src_mask: Optional[Tensor] = None,
                src_key_padding_mask: Optional[Tensor] = None) -> Tensor:
        
        src2 = self.self_attn(src, src, src, attn_mask=src_mask,
                              key_padding_mask=src_key_padding_mask)[0]
        src = src + self.dropout1(src2) # 1.
        src = self.norm1(src) # 2.
        src2 = self.linear2(self.dropout(self.activation(self.linear1(src)))) # 3.
        src = src + self.dropout2(src2) # 4-1.
        src = self.norm2(src) # 4-2.
        return src

TabError: inconsistent use of tabs and spaces in indentation (<ipython-input-64-2bc455c758bd>, line 25)